# AutoML: Train "the best" classifier model for the UCI Bank Marketing dataset. - With end-to-end MLFlow model experience.

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)
- The latest MLFlow packages:

    pip install azureml-mlflow

    pip install mlflow

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create an `AutoML classification Job` with the 'classification()' factory-fuction.
- Submit and run the AutoML classification job
- Obtaing the model and score predictions with it

**Motivations** - This notebook explains how to setup and run an AutoML classification job. This is one of the nine ML-tasks supported by AutoML. Other ML-tasks are 'regression', 'time-series forecasting', 'image classification', 'image object detection', 'nlp text classification', etc.

In this example we use the UCI Bank Marketing dataset to showcase how you can use AutoML for a classification problem. The classification goal is to predict if the client will subscribe to a term deposit with the bank.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [14]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.identity import AzureCliCredential
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient

from azure.ml._constants import AssetTypes
from azure.ml import automl
from azure.ml.automl import classification, ClassificationPrimaryMetrics, ClassificationModels
from azure.ml.entities import JobInput

## 1.2. Workspace details

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

By default, we try to use the by default workspace configuration (available out-of-the-box in Compute Instances) or from any Config.json file you might have copied into the folders structure.
If no Config.json is found, then you need to manually introduce the subscription_id, resource_group and workspace when creating MLClient .

In [15]:
# Using InteractiveBrowserCredential()
credential = InteractiveBrowserCredential()

# Using AzureCliCredential() instead InteractiveBrowserCredential()
# credential=AzureCliCredential()

ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP_NAME>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

Found the config file in: C:\Users\CESARDL.CESARDLSB2-MSFT\GitAMLRepos\azureml-examples-cesar-april-preview\sdk\jobs\automl-standalone-jobs\config.json


### Show Azure ML Workspace information

In [17]:
import pandas as pd

workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.connections._subscription_id
output["Resource Group"] = workspace.resource_group
output["Location"] = workspace.location
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

,
Workspace,cesardl-dist-training-eastus-ws
Subscription ID,381b38e9-9840-4719-a5a0-61d9585e1e91
Resource Group,cesardl-automl-eastus2euap-resgrp
Location,eastus


# 2. Configure and run the AutoML classification job
In this section we will configure and run the AutoML classification job.

## 2.1 Configure the job through the classification() factory function

### classification() parameters:

The `classification()` factory function allows user to configure AutoML for the classification task for the most common scenarios with the following properties.

- `target_column_name` - The name of the column to target for predictions. It must always be specified. This parameter is applicable to 'training_data', 'validation_data' and 'test_data'.
- `primary_metric` - The metric that AutoML will optimize for Classification model selection.
- `training_data` - The data to be used for training. It should contain both training feature columns and a target column. Optionally, this data can be split for segregating a validation or test dataset. 
You can use a registered MLTable in the workspace using the format '<mltable_name>:<version>' OR you can use a local file or folder as a MLTable. For e.g JobInput(mltable='my_mltable:1') OR JobInput(mltable=MLTable(local_path="./data"))
The parameter 'training_data' must always be provided.
- `compute` - The compute on which the AutoML job will run. In this example we are using a compute called 'cpu-cluster' present in the workspace. You can replace it any other compute in the workspace. 
- `name` - The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
- `experiment_name` - The name of the Experiment. An Experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment.

### set_limits() parameters:
This is an optional configuration method to configure limits parameters such as timeouts.     
    
- timeout_minutes - Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. This timeout includes setup, featurization and training runs but does not include the ensembling and model explainability runs at the end of the process since those actions need to happen once all the trials (children jobs) are done. If not specified, the default job's total timeout is 6 days (8,640 minutes). To specify a timeout less than or equal to 1 hour (60 minutes), make sure your dataset's size is not greater than 10,000,000 (rows times column) or an error results.

- trial_timeout_minutes - Maximum time in minutes that each trial (child job) can run for before it terminates. If not specified, a value of 1 month or 43200 minutes is used.
    
- max_trials - The maximum number of trials/runs each with a different combination of algorithm and hyperparameters to try during an AutoML job. If not specified, the default is 1000 trials. If using 'enable_early_termination' the number of trials used can be smaller.
    
- max_concurrent_trials - Represents the maximum number of trials (children jobs) that would be executed in parallel. It's a good practice to match this number with the number of nodes your cluster.
    
- enable_early_termination - Whether to enable early termination if the score is not improving in the short term. 
    

In [18]:
# Create MLTables for training dataset

my_training_data_input = JobInput(
    type=AssetTypes.MLTABLE, path="./data/training-mltable-folder"
)

# Remote MLTable definition
# my_training_data_input  = JobInput(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/Classification/Train")

In [23]:
# General job parameters
compute_name = "gpu-cluster" # "cesardl-d12v2-4-8-2022-v" 
max_trials = 5
exp_name = "dpv2-classifier-experiment"

In [24]:
# Create the AutoML classification job with the related factory-function.

classification_job = automl.classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name="y",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"my_custom_tag": "My custom value"}
)

# Limits are all optional
classification_job.set_limits(
    timeout=600,  # timeout_minutes
    trial_timeout=20,  # trial_timeout_minutes
    max_trials=max_trials,
    # max_concurrent_trials = 4,
    # max_cores_per_trial: -1,
    enable_early_termination=True,
)

# Training properties are optional
classification_job.set_training(
    blocked_models=[ClassificationModels.LOGISTIC_REGRESSION], 
    enable_onnx_compatible_models=True
)


## 2.2 Run the CommandJob
Using the `MLClient` created earlier, we will now run this CommandJob in the workspace.

In [25]:
# Submit the AutoML job 
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

Created job: ClassificationJob({'log_verbosity': <LogVerbosity.INFO: 'Info'>, 'task_type': <TaskType.CLASSIFICATION: 'Classification'>, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'display_name': '36a08260-5aa5-4425-a85b-8b782beb280d', 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': '36a08260-5aa5-4425-a85b-8b782beb280d', 'description': None, 'tags': {'my_custom_tag': 'My custom value'}, 'properties': {'mlflow.source.git.repoURL': 'git@github.com:Azure/azureml-examples.git', 'mlflow.source.git.branch': 'cesardl/april-sdk-preview', 'mlflow.source.git.commit': '0c98a5d87dc9fa34f87d9d18df36051555022ad8', 'azureml.git.dirty': 'True'}, 'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/cesardl-automl-eastus2euap-resgrp/providers/Microsoft.MachineLearningServices/workspaces/cesardl-dist-training-eastus-ws/jobs/36a08260-5aa5-4425-a85b-8b782beb280d', 'base_path': './', 'creation_context': <azure.ml._restclient.v2022_02_01_pr

### Wait until the AutoML job is finished
ml_client.jobs.stream(returned_job.name) waits until the specified job is finished

In [26]:
ml_client.jobs.stream(returned_job.name)

RunId: 36a08260-5aa5-4425-a85b-8b782beb280d
Web View: https://ml.azure.com/runs/36a08260-5aa5-4425-a85b-8b782beb280d?wsid=/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/cesardl-automl-eastus2euap-resgrp/workspaces/cesardl-dist-training-eastus-ws

Execution Summary
RunId: 36a08260-5aa5-4425-a85b-8b782beb280d
Web View: https://ml.azure.com/runs/36a08260-5aa5-4425-a85b-8b782beb280d?wsid=/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/cesardl-automl-eastus2euap-resgrp/workspaces/cesardl-dist-training-eastus-ws



In [ ]:
# Get a URL for the status of the job
returned_job.services["Studio"].endpoint

In [ ]:
print(returned_job.name)

# 3. Retrieve the Best Trial (Best Model's trial/run)
Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

## Initialize MLFlow Client
The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface. 
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

*IMPORTANT*, you need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow

    pip install mlflow

### Obtain the tracking URI for MLFlow

In [8]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/cesardl-automl-eastus2euap-resgrp/providers/Microsoft.MachineLearningServices/workspaces/cesardl-dist-training-eastus-ws


In [9]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/cesardl-automl-eastus2euap-resgrp/providers/Microsoft.MachineLearningServices/workspaces/cesardl-dist-training-eastus-ws


In [10]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

### Get the AutoML parent Job

In [12]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
#job_name = "b4e95546-0aa1-448e-9ad6-002e3207b4fc"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

RestException: RESOURCE_DOES_NOT_EXIST: Response: {'Error': {'Code': 'UserError', 'Severity': None, 'Message': 'Run b4e95546-0aa1-448e-9ad6-002e3207b4fc was not found', 'MessageFormat': None, 'MessageParameters': None, 'ReferenceCode': None, 'DetailsUri': None, 'Target': None, 'Details': [], 'InnerError': None, 'DebugInfo': {'Type': 'Microsoft.MachineLearning.MLflow.Contracts.Exceptions.ResourceDoesNotExistException', 'Message': 'Run b4e95546-0aa1-448e-9ad6-002e3207b4fc was not found', 'StackTrace': '   at Microsoft.MachineLearning.MLflow.Services.Utilities.ExceptionHandlingUtils.ExecuteAsync[T](Func`1 func) in /mnt/vss/_work/1/s/src/azureml-api/src/MLflow/Services/Utilities/ExceptionHandlingUtils.cs:line 41\n   at Microsoft.MachineLearning.MLflow.Services.TrackingService.GetRunFromWorkspace(WorkspaceContext2 workspaceContext, String runId) in /mnt/vss/_work/1/s/src/azureml-api/src/MLflow/Services/TrackingService.cs:line 546\n   at Microsoft.MachineLearning.MLflow.Services.TrackingServ

In [13]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

NameError: name 'mlflow_parent_run' is not defined

## Get the AutoML best child run

In [ ]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

## Get best model run's metrics

Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [ ]:
import pandas as pd

pd.DataFrame(best_run.data.metrics, index=[0]).T

## Download the best model locally

Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [ ]:
# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [ ]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

In [ ]:
import os

# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

# Next Step: Load the best model and try predictions

Loading the models locally assume that you are running the notebook in an environment compatible with the model. The list of dependencies that is expected by the model is specified in the MLFlow model produced by AutoML (in the 'conda.yaml' file within the mlflow-model folder).

Since the AutoML model was trained remotelly in a different environment with different dependencies to your current local conda environment where you are running this notebook, if you want to load the model you have several options:

1. A recommended way to locally load the model in memory and try predictions is to create a new/clean conda environment with the dependencies specified in the conda.yml file within the MLFlow model's folder, then use MLFlow to load the model and call .predict() as explained in the notebook **mlflow-model-local-inference-test.ipynb** in this same folder.

2. You can install all the packages/dependencies specified in conda.yml into your current conda environment you used for using Azure ML SDK and AutoML. MLflow SDK also have a method to install the dependencies in the current environment. However, this option could have risks of package version conflicts depending on what's installed in your current environment.

3. You can also use: mlflow models serve -m 'xxxxxxx'

# Next Steps
You can see further examples of other AutoML tasks such as Image-Classification, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.